In [ ]:
import cv2 #import the library 
import numpy as np

import matplotlib.pyplot as plt


import tensorflow as tf
from statistics import *
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import img_to_array

cap = cv2.VideoCapture(0) # capture the webcam
cap.set(3, 640) ## define the dimensions
cap.set(4, 480)



In [ ]:
def getContours(img,original_img):
    
    contours,hierachy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        
        area = cv2.contourArea(cnt)
       
        if area > 60000:
            cv2.drawContours(original_img,cnt,-1,(0,255,0),2)
            #print(area)
            
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            ax = approx.item(0)
            ay = approx.item(1)
            bx = approx.item(2)
            by = approx.item(3)
            cx = approx.item(4)
            cy = approx.item(5)
            dx = approx.item(6)
            dy = approx.item(7)
            
            width,height= 900,900

            pts1 = np.float32([[bx,by],[ax,ay],[cx,cy],[dx,dy]])
            pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])

            matrix = cv2.getPerspectiveTransform(pts1,pts2)
            img_perspective = cv2.warpPerspective(original_img,matrix,(width,height))
            img_corners = cv2.cvtColor(img_perspective,cv2.COLOR_BGR2GRAY)
            
            # Turn into a binary image
            for x in range(0, 900):
                for y in range(0, 900):
                    if img_corners[x][y]<100:
                        img_corners[x][y]=0
                    else:
                        img_corners[x][y]=255
                        
            cv2.imshow('Corners',img_corners)
            
            classify(img_corners)
            
      

In [ ]:
def classify(Img):
     
    DATADROP = "C://IMG//"
    crop_val = 20
    
    
    for i in range(0, 9):
        for j in range(0, 9):
            
                
            J = j+1
            I = i+1
            cell = Img[I*100 - 100 + crop_val : I*100 - crop_val , J*100 - 100 + crop_val : J*100 - crop_val]

            img_canny = cv2.Canny(cell, 50, 50)

            contours, hierachy = cv2.findContours(img_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            for cnt in contours:
                area = cv2.contourArea(cnt)

                if area > 5:

                    peri = cv2.arcLength(cnt, True)
                    approx = cv2.approxPolyDP(cnt, 0.02*peri, True)

                    x,y,w,h = cv2.boundingRect(approx)

                    image_rect = cell[y:y+h, x:x+w]
                    image_rect = cv2.resize(image_rect, (100, 100))

                    image_num = img_to_array(image_rect)

                    image_num = np.array(image_num).reshape(-1, 100 , 100,1)
                    image_num = image_num.astype('float32')
                    image_num = image_num / 255.0

                    model = tf.keras.models.load_model('sudoku_model_ytb.h5')
                    digit = int(model.predict_classes(image_num))
                    prediction = model.predict(image_num)
                    prob = np.amax(prediction)
                    plt.imshow(image_rect, cmap='gray')
                    plt.show()
                    print("Detected: ",digit)
                    print("Probability: ",prob)
                

In [ ]:
while True:
    success, img = cap.read()
    # Turning the original image to canny.
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),3)
    imgCanny = cv2.Canny(imgBlur,50,50)
    img_copy = img.copy()
    
    
    
    
    img_contours = img.copy()
    # Getting the countours
    getContours(imgCanny, img_copy)
    
    cv2.imshow('Webcam',img_copy)
    
    
    
    #cv2.imshow('Webcam',img_contours)
    if cv2.waitKey(1) & 0xff == ord('q'):
        cv2.destroyAllWindows()
        break
        